## Import Libraries


In [1]:
#Import Pennylane

import pennylane as qml

#Import Pytorch

import torch
from torch.utils.data import DataLoader

#Import other libraries

import numpy as np
from numpy.random import randn, randint
import os
import time
import math
import matplotlib.pyplot as plt
from matplotlib.ticker import PercentFormatter


OSError: [WinError 126] The specified module could not be found. Error loading "C:\Users\jonas\Anaconda3\envs\q_bootcmap\lib\site-packages\torch\lib\cudnn_cnn_infer64_8.dll" or one of its dependencies.

In [2]:
!pip3 install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118

Looking in indexes: https://download.pytorch.org/whl/cu118
  Using cached https://download.pytorch.org/whl/cu118/torch-2.3.0%2Bcu118-cp310-cp310-win_amd64.whl (2673.0 MB)
  Using cached https://download.pytorch.org/whl/cu118/torchvision-0.18.0%2Bcu118-cp310-cp310-win_amd64.whl (4.9 MB)
  Using cached https://download.pytorch.org/whl/cu118/torchaudio-2.3.0%2Bcu118-cp310-cp310-win_amd64.whl (4.0 MB)


ERROR: Could not install packages due to an OSError: [WinError 5] Access is denied: 'C:\\Users\\jonas\\Anaconda3\\envs\\q_bootcmap\\Lib\\site-packages\\torch\\lib\\asmjit.dll'
Consider using the `--user` option or check the permissions.



# Idea

The option pricing project had two main parts that used quantum algorithms: loading the underlying distributions, and amplitude estimation to calculate the payoff. In order to solve the former problem, Qiskit came up with the idea of using qGANs for loading the random distributions (see following tutorial):

https://qiskit.org/documentation/machine-learning/tutorials/04_qgans_for_loading_random_distributions.html

This is to a certain extent a question of quantum state preparation, where we are trying to find a quantum circuit that can prepare the state that describes the underlying distribution reflected in reality.

However, in the process of replicating Qiskit's solution and trying to improve on it, I realized that there was no need for a discriminator. Fundamentally, GANs in general are used for generative purposes, for unsupervised learning. However, in this case, we had the ideal result already: the distribution. Thus, it seemed to me that all we needed to do was to fit the results running the QNN with just one quantum layer to the distribution (which honestly makes it not exactly a NN), bypassing the need for a discriminator.

# Data

To demonstrate, we use a lognormal data-set. The data preparation phase consists of vectorizing (or binning) the data as with a quantum computer, we have a limited number of values we can use ($2^n$ where n is the number of qubits). For this particular example, we use 4 qubits, so we use 16 bins.

In [ ]:
n_qubits = 4
n_bins = 2 ** 4

### Create the data

In [ ]:
# Number training data samples
N = 10000

mu = 1
sigma = 1
data = np.random.lognormal(mean=mu, sigma=sigma, size=N)
np.random.shuffle(data)

### Put the data into bins

In [ ]:
data_pre = np.round(data)
data = data_pre[data_pre <= n_bins]

bins = np.linspace(0, (n_bins - 1) , num= n_bins )
bin_indices = np.digitize(data, bins) - 1
data_temp = ((np.arange(n_bins) == bin_indices[:,None]).astype(int))

In [ ]:
plt.hist(bin_indices, bins = n_bins)

In [ ]:
data_dist = np.sum(data_temp, axis = 0)/10000

In [ ]:
data_dist

In [ ]:
fig = plt.figure()
ax = fig.add_axes([0,0,1,1])
y = range(n_bins)
ax.bar(y, data_dist, alpha=0.5)
plt.show()

The variable data_dist above gives the distribution that we want to model.

# Model

We use Pennylane and PyTorch for this. Note that with Pennylane, it is easy to adapt to interface (like Braket or Qiskit) since we are tapping on their quantum computers.

In [ ]:
dev = qml.device("default.qubit", wires= n_qubits, shots = 10000)

In [ ]:
# Number of variational layers
q_depth = 2

Here we train using variational methods. 

Three parts of note:
- Initial Distribution
- Variational Quantum Circuit
- Measurement


In [ ]:
@qml.qnode(dev, interface='torch')
def qnode(inputs, weights):
    #qml.templates.AngleEmbedding(inputs, wires=range(n_qubits), rotation='Y')
    #qml.templates.BasicEntanglerLayers(weights, wires=range(n_qubits),rotation= qml.RY)
    
    # Init distribution
    for a in range(n_qubits):
        qml.Hadamard(wires=a)
    
    # Variational circuit
    for i in range(q_depth):
        for j in range(n_qubits):
            qml.RY(weights[2*(i*n_qubits + j) ], wires=j)
            qml.RZ(weights[2*(i*n_qubits + j) + 1], wires=j)
        for l in range(n_qubits):
            if (l == (n_qubits - 1)):
                qml.CNOT(wires=[l,0])
            else:
                qml.CNOT(wires=[l,l+1])
    
    for k in range(n_qubits):
        qml.RY(weights[(2*q_depth * n_qubits) + k ], wires=k)
        qml.RZ(weights[(2*q_depth * n_qubits) + k + 1], wires=k)
    
    # Measurement
    return qml.probs(wires=range(n_qubits))

In [ ]:
n_args = 2*(q_depth +1) * n_qubits
weight_shapes = {"weights": n_args}

In [ ]:
# Visualize the circuit
print(qml.draw(qnode)(inputs = 0, weights = np.random.rand(n_args)))

In [ ]:
# Create the model 
qlayer = qml.qnn.TorchLayer(qnode, weight_shapes)
layers = [qlayer]
model = torch.nn.Sequential(*layers)

In [ ]:
# Define optimizer and loss function
opt = torch.optim.Adam(model.parameters(), lr=0.001, betas=(0.5, 0.999))
loss = torch.nn.MSELoss()

We take a look at the initial randomized distribution of the quantum state generated by the quantum circuit.

In [ ]:
noise = torch.normal(0, 1, size=(1,4))

# predict outputs
outputs = model(noise[0])
outputs

In [ ]:
fig = plt.figure()
ax = fig.add_axes([0,0,1,1])
y = range(16)
ax.bar(y, outputs.detach().numpy() )
plt.show()

# Training

In [ ]:
# Decide number of epochs
n_epochs=2000

# Ideal probability distribution, in correct format
data_dist_t = torch.from_numpy(data_dist.astype("float32"))

In [ ]:
start = time.time()
seed = torch.normal(0, 1, size=(1, 4))
for i in range(n_epochs):
    if (i % 100 == 0): 
        test_data = model(seed[0])
        fig = plt.figure()
        ax = fig.add_axes([0,0,1,1])
        y = range(16)
        ax.bar(y, test_data.detach().numpy() )
        plt.show()

    # new_dist = g_model.predict(seed)
    opt.zero_grad()

    loss_evaluated = loss(model(seed[0]), data_dist_t)
    loss_evaluated.backward()

    opt.step()

    if (i % 100 == 0): 
        print((i+1), loss_evaluated)

end = time.time()
print("Execution time", end - start)

# Visualizations

Compare the real data to the one generated by the quantum state.

In [ ]:
fig = plt.figure()
ax = fig.add_axes([0,0,1,1])
y = range(16)
ax.bar(y, model(noise[0]).detach().numpy(), alpha=0.5, label = 'QNN' )
ax.bar(y, data_dist_t.detach().numpy(), alpha=0.5, label = 'Real' )
ax.legend()
plt.title("Probability distributions generated vs Real")
plt.show()